## インストール

In [ ]:
!pip install -q --no-warn-conflicts "colab-ai-bridge[langchain] @ git+https://github.com/drillan/colab-ai-bridge"

## 使い方

### 基本的な使い方

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel

# モデルの作成（セットアップは自動で完了します）
model = ColabLangChainModel()

# モデルの実行
response = model.invoke("フランスの首都は？")
print(response.content)

### モデルの選択

利用可能なモデルを確認：

In [ ]:
from google.colab import ai

ai.list_models()

特定のモデルを使用：

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel

# Gemini 2.5 Flash Liteを使用
model = ColabLangChainModel("google/gemini-2.5-flash-lite")

### 構造化出力

LangChain の Messages API を使用して構造化されたプロンプトを作成：

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel
import json


class City(BaseModel):
    name: str
    country: str
    population: int


model = ColabLangChainModel()

messages = [
    SystemMessage(content="あなたは構造化されたデータを JSON 形式で返すアシスタントです。"),
    HumanMessage(
        content=f"""東京について、以下の JSON スキーマに従って情報を返してください：
{City.model_json_schema()}

JSON のみを返してください。"""
    ),
]

response = model.invoke(messages)
city_data = json.loads(response.content)
city = City(**city_data)
print(f"{city.name}, {city.country}, 人口: {city.population:,}")

## サンプル

### サンプル 1: リスト型フィールドを持つ構造化出力

リスト型フィールドを含むデータモデル

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
import json


class EmailAddress(BaseModel):
    email: str = Field(pattern=r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$")
    domain: str


model = ColabLangChainModel()

messages = [
    SystemMessage(content="あなたは構造化されたデータを JSON 形式で返すアシスタントです。"),
    HumanMessage(
        content=f"""example@gmail.com のメールアドレス情報を以下のスキーマで抽出してください：
{EmailAddress.model_json_schema()}

JSON のみを返してください。"""
    ),
]

response = model.invoke(messages)
email_data = json.loads(response.content)
email = EmailAddress(**email_data)
print(f"メール: {email.email}")
print(f"ドメイン: {email.domain}")

### サンプル 2: システムプロンプト付きモデル

役割を持たせたモデルで専門的な回答を得る

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel
import json


class Book(BaseModel):
    title: str
    author_name: str
    year: int
    genres: list[str]


model = ColabLangChainModel()

messages = [
    SystemMessage(content="あなたは文学の専門家です。本の情報を JSON 形式で提供してください。"),
    HumanMessage(
        content=f"""村上春樹の『ノルウェイの森』について、以下のスキーマで情報を返してください：
{Book.model_json_schema()}

JSON のみを返してください。"""
    ),
]

response = model.invoke(messages)
book_data = json.loads(response.content)
book = Book(**book_data)
print(f"タイトル: {book.title}")
print(f"著者: {book.author_name}")
print(f"出版年: {book.year}")
print(f"ジャンル: {', '.join(book.genres)}")

### サンプル 3: プロンプトテンプレートの使用

LangChain のプロンプトテンプレートで動的な値を渡す

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from langchain_core.prompts import ChatPromptTemplate

model = ColabLangChainModel()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは日本史の専門家です。歴史的な事実を正確に、わかりやすく説明してください。"),
        ("human", "{topic}について簡潔に説明してください"),
    ]
)

chain = prompt | model
result = chain.invoke({"topic": "関ヶ原の戦い"})
print(result.content)

### サンプル 4: 温度パラメータの調整

temperature を変更して創造的/正確な出力を制御

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel
from colab_ai_bridge.core.config import ModelConfig
from langchain_core.prompts import ChatPromptTemplate
from dataclasses import dataclass


@dataclass
class UserProfile:
    name: str
    age: int
    interests: list[str]


model = ColabLangChainModel()

user = UserProfile(name="太郎", age=25, interests=["プログラミング", "機械学習", "読書"])

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーのプロフィールに基づいて、パーソナライズされた提案をしてください。"),
        (
            "human",
            """ユーザー情報:
- 名前: {name}
- 年齢: {age}
- 興味: {interests}

おすすめの学習リソースを3つ教えてください""",
        ),
    ]
)

chain = prompt | model
result = chain.invoke(
    {"name": user.name, "age": user.age, "interests": ", ".join(user.interests)}
)
print(result.content)

### サンプル 5: 温度パラメータによる出力の違い

異なる温度設定での出力比較

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel

# 低温度（正確な出力）
config_precise = ModelConfig(temperature=0.3)
model_precise = ColabLangChainModel(config=config_precise)

# 高温度（創造的な出力）
config_creative = ModelConfig(temperature=1.0)
model_creative = ColabLangChainModel(config=config_creative)

prompt = "AIについて一文で説明してください"

print("【正確な出力 (temperature=0.3)】")
result1 = model_precise.invoke(prompt)
print(result1.content)
print()

print("【創造的な出力 (temperature=1.0)】")
result2 = model_creative.invoke(prompt)
print(result2.content)

### サンプル 6: ストリーミング出力

リアルタイムでレスポンスをストリーミング

In [ ]:
from colab_ai_bridge.langchain import ColabLangChainModel

model = ColabLangChainModel()

print("ストリーミング出力:")
for chunk in model.stream("日本の四季について説明してください"):
    print(chunk.content, end="", flush=True)
print()